# Kite Connect Testing Notebook

Test suite for Kite API integration, position analysis, and trading operations.

## 1. Import Dependencies and Initialize Kite

In [1]:
# In your main_strategy.py
from pprint import pprint
import pandas as pd
import numpy as np
from datetime import datetime, date

In [2]:
# Reload the module to get fresh kite object with new token
import importlib
import load_kite_from_access
#importlib.reload(load_kite_from_access)
kite = load_kite_from_access.kite

# Check kite status and get UserID
print(f"Kite object: {kite}")
print(f"Type: {type(kite)}")
if kite:
    try:
        profile = kite.profile()
        print(f"\n✅ UserID: {profile['user_id']}")
        print(f"✅ User Name: {profile['user_name']}")
        print(f"✅ Email: {profile['email']}")
    except Exception as e:
        print(f"Error getting profile: {e}")
else:
    print("\n❌ Kite object is None - session not established")
    print("Please ensure you have run config.py to generate access token")

✅ Connected: Indhuja . (ID: XJY521)
Kite object: <kiteconnect.connect.KiteConnect object at 0x000001E095240EC0>
Type: <class 'kiteconnect.connect.KiteConnect'>

✅ UserID: XJY521
✅ User Name: Indhuja .
✅ Email: sathyakumarnandakumar@gmail.com


## 2. Get User Profile

In [11]:

# The kite object is already initialized and validated!
if kite:
    print(kite)
    print("Kite connection successful!")
    profile = kite.profile()
    user_id = profile['user_id']
    user_name = profile['user_name']
    print(f"Connected as User ID: {user_id}")
    print(f"User Name: {user_name}")

## 3. Fetch and Group Instruments (NSE + NFO)

## 4. Export Instruments to CSV Files

In [35]:
# Standardize and format all dataframes for consistency
print("="*80)
print("STANDARDIZING DATA FORMATS")
print("="*80)

# Helper function to standardize datetime columns
def standardize_dataframe(df, name="DataFrame"):
    df_clean = df.copy()
    
    # Convert date columns to datetime64
    date_columns = ['expiry', 'last_date']
    for col in date_columns:
        if col in df_clean.columns:
            df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')
    
    # Ensure numeric columns are proper types
    numeric_columns = ['strike', 'tick_size', 'lot_size']
    for col in numeric_columns:
        if col in df_clean.columns:
            df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
    
    # String columns should be proper strings (including tokens)
    string_columns = ['tradingsymbol', 'name', 'exchange', 'segment', 'instrument_type', 
                      'instrument_token', 'exchange_token']
    for col in string_columns:
        if col in df_clean.columns:
            df_clean[col] = df_clean[col].astype(str)
    
    print(f"✅ {name}: {len(df_clean):,} rows standardized")
    return df_clean

# Standardize all NSE/NFO dataframes
print("\n📊 NSE/NFO DataFrames:")
df_all = standardize_dataframe(df_all, "df_all")
df_equity = standardize_dataframe(df_equity, "df_equity")
df_index = standardize_dataframe(df_index, "df_index")
df_futures = standardize_dataframe(df_futures, "df_futures")
df_options_ce = standardize_dataframe(df_options_ce, "df_options_ce")
df_options_pe = standardize_dataframe(df_options_pe, "df_options_pe")

# Standardize MCX dataframe
print("\n📊 MCX DataFrames:")
df_mcx = standardize_dataframe(df_mcx, "df_mcx")

# Create and standardize MCX filtered dataframes
df_mcx_futures = df_mcx[df_mcx['instrument_type'] == 'FUT'].copy()
df_mcx_ce = df_mcx[df_mcx['instrument_type'] == 'CE'].copy()
df_mcx_pe = df_mcx[df_mcx['instrument_type'] == 'PE'].copy()

print(f"✅ df_mcx_futures: {len(df_mcx_futures):,} rows")
print(f"✅ df_mcx_ce: {len(df_mcx_ce):,} rows")
print(f"✅ df_mcx_pe: {len(df_mcx_pe):,} rows")

print("\n" + "="*80)
print("DATA STANDARDIZATION COMPLETE")
print("="*80)
print("All dataframes now have:")
print("  - DateTime columns in datetime64 format")
print("  - Numeric columns (strike, tick_size, lot_size) in proper numeric types")
print("  - String columns (including instrument_token, exchange_token) as strings")
print("  - Ready for Parquet export!")

STANDARDIZING DATA FORMATS

📊 NSE/NFO DataFrames:
✅ df_all: 47,997 rows standardized
✅ df_equity: 9,068 rows standardized
✅ df_index: 134 rows standardized
✅ df_futures: 633 rows standardized
✅ df_options_ce: 19,137 rows standardized
✅ df_options_pe: 19,025 rows standardized

📊 MCX DataFrames:
✅ df_mcx: 73,774 rows standardized
✅ df_mcx_futures: 145 rows
✅ df_mcx_ce: 36,809 rows
✅ df_mcx_pe: 36,809 rows

DATA STANDARDIZATION COMPLETE
All dataframes now have:
  - DateTime columns in datetime64 format
  - Numeric columns (strike, tick_size, lot_size) in proper numeric types
  - String columns (including instrument_token, exchange_token) as strings
  - Ready for Parquet export!


In [34]:
# Save all DataFrames to Parquet files for efficient storage
import os

# Create a folder for the data
output_folder = "instruments_data"
os.makedirs(output_folder, exist_ok=True)

# Save each DataFrame to Parquet (data already standardized in previous cell)
files_created = []

# NSE + NFO instruments
df_all.to_parquet(f"{output_folder}/df_all.parquet", engine='pyarrow')
files_created.append(f"{output_folder}/df_all.parquet")

df_equity.to_parquet(f"{output_folder}/equity.parquet", engine='pyarrow')
files_created.append(f"{output_folder}/equity.parquet")

df_index.to_parquet(f"{output_folder}/indices.parquet", engine='pyarrow')
files_created.append(f"{output_folder}/indices.parquet")

df_futures.to_parquet(f"{output_folder}/futures.parquet", engine='pyarrow')
files_created.append(f"{output_folder}/futures.parquet")

df_options_ce.to_parquet(f"{output_folder}/options_ce.parquet", engine='pyarrow')
files_created.append(f"{output_folder}/options_ce.parquet")

df_options_pe.to_parquet(f"{output_folder}/options_pe.parquet", engine='pyarrow')
files_created.append(f"{output_folder}/options_pe.parquet")

# MCX dataframes (already created and standardized in previous cell)
df_mcx.to_parquet(f"{output_folder}/mcx_all.parquet", engine='pyarrow')
files_created.append(f"{output_folder}/mcx_all.parquet")

df_mcx_futures.to_parquet(f"{output_folder}/mcx_futures.parquet", engine='pyarrow')
files_created.append(f"{output_folder}/mcx_futures.parquet")

df_mcx_ce.to_parquet(f"{output_folder}/mcx_options_ce.parquet", engine='pyarrow')
files_created.append(f"{output_folder}/mcx_options_ce.parquet")

df_mcx_pe.to_parquet(f"{output_folder}/mcx_options_pe.parquet", engine='pyarrow')
files_created.append(f"{output_folder}/mcx_options_pe.parquet")

print("✅ Parquet files created successfully!")
print(f"\n📊 Summary:")
print(f"   - NSE/NFO Total: {len(df_all):,} instruments")
print(f"   - MCX Total: {len(df_mcx):,} instruments")
print(f"   - MCX Futures: {len(df_mcx_futures):,} instruments")
print(f"   - MCX CE: {len(df_mcx_ce):,} instruments")
print(f"   - MCX PE: {len(df_mcx_pe):,} instruments")

print("\n📁 Files saved to:")
for file in files_created:
    full_path = os.path.abspath(file)
    file_size = os.path.getsize(full_path) / (1024 * 1024)  # Size in MB
    print(f"   - {file} ({file_size:.2f} MB)")
    
print("\n💡 To view in Python:")
print("   import pandas as pd")
print("   df = pd.read_parquet('instruments_data/equity.parquet')")

✅ Parquet files created successfully!

📊 Summary:
   - NSE/NFO Total: 47,997 instruments
   - MCX Total: 73,774 instruments
   - MCX Futures: 145 instruments
   - MCX CE: 36,809 instruments
   - MCX PE: 36,809 instruments

📁 Files saved to:
   - instruments_data/df_all.parquet (1.12 MB)
   - instruments_data/equity.parquet (0.35 MB)
   - instruments_data/indices.parquet (0.01 MB)
   - instruments_data/futures.parquet (0.03 MB)
   - instruments_data/options_ce.parquet (0.53 MB)
   - instruments_data/options_pe.parquet (0.53 MB)
   - instruments_data/mcx_all.parquet (1.42 MB)
   - instruments_data/mcx_futures.parquet (0.01 MB)
   - instruments_data/mcx_options_ce.parquet (0.94 MB)
   - instruments_data/mcx_options_pe.parquet (0.94 MB)

💡 To view in Python:
   import pandas as pd
   df = pd.read_parquet('instruments_data/equity.parquet')


## 5. Create Instrument Tree (Nested Dictionary)

In [24]:
# Create a nested dictionary: { name: { type: dataframe } }
# This groups by name first, then by instrument_type within each name
instrument_tree = {
    name: {inst_type: data for inst_type, data in name_group.groupby('instrument_type')}
    for name, name_group in df_all.groupby('name')
}



In [ ]:
print(f"✅ Created instrument_tree with {len(instrument_tree)} unique instrument names")
print(f"Example: instrument_tree['BSE'] contains: {list(instrument_tree.get('BSE', {}).keys())}")

# Usage:
BSE = instrument_tree['BSE']['FUT']
pprint(BSE)

✅ Created instrument_tree with 7582 unique instrument names
Example: instrument_tree['ADANIENT'] contains: ['CE', 'EQ', 'FUT', 'PE']
     instrument_token exchange_token tradingsymbol name  last_price  \
9306         15174914          59277   BSE26FEBFUT  BSE         0.0   
9307         13269762          51835   BSE26MARFUT  BSE         0.0   
9308         17084162          66735   BSE26APRFUT  BSE         0.0   

         expiry  strike  tick_size  lot_size instrument_type  segment exchange  
9306 2026-02-24     0.0        0.1       375             FUT  NFO-FUT      NFO  
9307 2026-03-30     0.0        0.1       375             FUT  NFO-FUT      NFO  
9308 2026-04-28     0.0        0.1       375             FUT  NFO-FUT      NFO  


## 5. Option Chain Analysis

In [ ]:
# Create a comprehensive lookup dictionary for symbol metadata
# Format: { 'SYMBOL_NAME': { 'exchange': 'EXCHANGE', 'expiries': [date1, date2, ...] } }

symbol_lookup = {}

# # 1. Ensure we have MCX data
# if 'df_mcx' not in locals() and 'instruments_mcx' in locals():
#     print("Creating df_mcx from instruments_mcx...")
#     df_mcx = pd.DataFrame(instruments_mcx)
#     # Standardize expiry
#     if 'expiry' in df_mcx.columns:
#          df_mcx['expiry'] = pd.to_datetime(df_mcx['expiry'], errors='coerce')


# 2. Process NSE/NFO/MCX Instruments (from df_all)
print("Building lookup for NSE/NFO symbols...")

if 'df_all' in locals():
    for name, group in df_all.groupby('name'):
        # Determine the primary exchange for trading (prefer NFO if available, else NSE/Indices)
        exchanges = group['exchange'].unique()
        
        if 'NFO' in exchanges:
            primary_exchange = 'NFO'
        elif 'NSE' in exchanges:
            primary_exchange = 'NSE'
        else:
            primary_exchange = exchanges[0]
            
        # extract valid expiries
        expiries = sorted(group['expiry'].dropna().unique())
        
        symbol_lookup[name] = {
            'exchange': primary_exchange,
            'expiries': expiries
        }
else:
    print("⚠️ df_all not found in local variables.")

# 3. Process MCX Instruments (from df_mcx)
if 'df_mcx' in locals():
    print("Building lookup for MCX symbols...")
    for name, group in df_mcx.groupby('name'):
        # MCX usually has expiries for everything
        expiries = sorted(group['expiry'].dropna().unique())
        
        symbol_lookup[name] = {
            'exchange': 'MCX',
            'expiries': expiries
        }
else:
    print("⚠️ df_mcx not found. MCX symbols will be missing from lookup.")

print(f"\n✅ Symbol Lookup Created for {len(symbol_lookup)} symbols")

# --- TEST VARIFICATION ---
print("-" * 50)
print("TESTING LOOKUP:")
test_symbols = ['GOLDM', 'CRUDEOIL', 'ADANIENT', 'NIFTY', 'RELIANCE']

for sym in test_symbols:
    if sym in symbol_lookup:
        entry = symbol_lookup[sym]
        exp_count = len(entry['expiries'])
        first_expiry = entry['expiries'][0].date() if exp_count > 0 else "None"
        print(f"✅ {sym:<10} | Exch: {entry['exchange']:<5} | Expiries: {exp_count} (Next: {first_expiry})")
    else:
        print(f"❌ {sym:<10} | Not found in lookup")
print("-" * 50)

Building lookup for NSE/NFOMCX symbols...
Building lookup for MCX symbols...

✅ Symbol Lookup Created for 7626 symbols
--------------------------------------------------
GOLDM:    {'exchange': 'MCX', 'expiries': [Timestamp('2026-02-05 00:00:00'), Timestamp('2026-02-26 00:00:00'), Timestamp('2026-03-05 00:00:00'), Timestamp('2026-03-26 00:00:00'), Timestamp('2026-04-03 00:00:00'), Timestamp('2026-05-05 00:00:00'), Timestamp('2026-06-05 00:00:00'), Timestamp('2026-07-03 00:00:00')]}
ADANIENT: {'exchange': 'NFO', 'expiries': [Timestamp('2026-02-24 00:00:00'), Timestamp('2026-03-30 00:00:00'), Timestamp('2026-04-28 00:00:00')]}
NIFTY:    {'exchange': 'NFO', 'expiries': [Timestamp('2026-02-03 00:00:00'), Timestamp('2026-02-10 00:00:00'), Timestamp('2026-02-17 00:00:00'), Timestamp('2026-02-24 00:00:00'), Timestamp('2026-03-02 00:00:00'), Timestamp('2026-03-30 00:00:00'), Timestamp('2026-04-28 00:00:00'), Timestamp('2026-06-30 00:00:00'), Timestamp('2026-09-29 00:00:00'), Timestamp('2026-12-

In [13]:
# Function to get all available expiries for a symbol
def get_all_expiries(kite, symbol, exchange=None):
    """
    Get all available expiry dates for a given symbol using the pre-computed lookup table.
    
    Args:
        kite: KiteConnect instance (kept for compatibility, though not used if lookup exists)
        symbol: Base symbol name (e.g., "NIFTY", "GOLDM")
        exchange: Exchange name (optional, will be auto-detected from lookup)
    
    Returns:
        List of expiry dates sorted in ascending order
    """
    try:
        # 1. Try to use the pre-computed symbol_lookup dictionary
        if 'symbol_lookup' in globals() and symbol in symbol_lookup:
            # print(f"✅ Found '{symbol}' in lookup table (Exchange: {symbol_lookup[symbol]['exchange']})")
            return symbol_lookup[symbol]['expiries']
            
        # 2. Fallback: Fetch from kite if lookup not available/symbol not found
        print(f"⚠️ Symbol '{symbol}' not found in lookup table. Fetching from API...")
        
        # Determine exchange if not provided
        if not exchange:
            exchange = "NFO" # Default
            
        instruments = kite.instruments(exchange)
        matching = [i for i in instruments if i['name'] == symbol and i['expiry']]
        expiries = sorted(list(set(i['expiry'] for i in matching)))
        
        return expiries
        
    except Exception as e:
        print(f"Error fetching expiries: {str(e)}")
        return []

# Example usage help
print("Function get_all_expiries(kite, symbol) updated to use symbol_lookup table.")

Function get_all_expiries(kite, symbol) updated to use symbol_lookup table.


In [14]:
get_all_expiries(kite, symbol= 'GOLDM')

[Timestamp('2026-02-05 00:00:00'),
 Timestamp('2026-02-26 00:00:00'),
 Timestamp('2026-03-05 00:00:00'),
 Timestamp('2026-03-26 00:00:00'),
 Timestamp('2026-04-03 00:00:00'),
 Timestamp('2026-05-05 00:00:00'),
 Timestamp('2026-06-05 00:00:00'),
 Timestamp('2026-07-03 00:00:00')]

In [ ]:
print(f"✅ Created instrument_tree with {len(instrument_tree)} unique instrument names")
print(f"Example: instrument_tree['ADANIENT'] contains: {list(instrument_tree.get('BSE', {}).keys())}")      

In [18]:
def build_option_chain(symbol, expiry):
    """
    Build option chain for a symbol and expiry using global dataframes (df_all/df_mcx) and symbol_lookup.
    Automatically selects the correct exchange and DataFrame.
    
    Args:
        symbol (str): Base symbol (e.g. 'NIFTY', 'GOLDM')
        expiry (str/date): Expiry date
        
    Returns:
        DataFrame: Option chain with columns [tradingsymbol, strike, instrument_type, expiry, exchange, etc from source]
    """
    # 1. Select the source Dataframe based on lookup or default
    source_df = None
    exchange = "NFO" # Default fallback
    
    if 'symbol_lookup' in globals() and symbol in symbol_lookup:
        exchange = symbol_lookup[symbol]['exchange']
        
        # Decide which global DF to use
        if exchange == 'MCX' and 'df_mcx' in globals():
            source_df = globals()['df_mcx']
        elif 'df_all' in globals():
            source_df = globals()['df_all']
            
    else:
        # Fallback if lookup fails but df_all exists (assume NFO/NSE)
        if 'df_all' in globals():
             source_df = globals()['df_all']
            
    if source_df is None:
        print("❌ Error: No instruments data found (df_all/df_mcx missing in globals)")
        return pd.DataFrame()
        
    # 2. Filter Data
    # Ensure expiry is compatible (Timestamp)
    try:
        expiry = pd.to_datetime(expiry)
    except:
        pass
        
    # Perform filtering
    # We look for: Name matches symbol AND Expiry matches AND Instrument is CE or PE
    mask = (
        (source_df['name'] == symbol) &
        (source_df['expiry'] == expiry) &
        (source_df['instrument_type'].isin(['CE', 'PE']))
    )
    
    chain = source_df[mask].copy()
    
    if chain.empty:
        print(f"⚠️ No options found for {symbol} expiring {expiry.date()} (Exchange: {exchange})")
        return pd.DataFrame()
        
    # Make sure we sort by strike for better view
    cols_priority = ['tradingsymbol', 'strike', 'instrument_type', 'expiry', 'lot_size', 'exchange', 'instrument_token']
    available_cols = [c for c in cols_priority if c in chain.columns]
    
    return chain[available_cols].sort_values('strike').reset_index(drop=True)


def enrich_with_market_data(kite, option_chain_df):
    """
    Fetch market depth/Limit/OI for the option chain dataframe
    """
    if option_chain_df.empty:
        return option_chain_df

    # 1. Construct Exchange:Symbol List for kite.quote()
    # Uses 'exchange' column if available (from build_option_chain), else defaults to 'NFO'
    symbols_to_quote = []
    
    if 'exchange' in option_chain_df.columns:
        # Vectorized string creation is faster
        symbols_to_quote = (option_chain_df['exchange'] + ":" + option_chain_df['tradingsymbol']).tolist()
    else:
        # Fallback logic
        symbols_to_quote = ["NFO:" + s for s in option_chain_df['tradingsymbol']]

    # 2. Fetch Quotes (Batch)
    try:
        quotes = kite.quote(symbols_to_quote)
    except Exception as e:
        print(f"Error fetching quotes: {e}")
        return option_chain_df

    # 3. Enrich DataFrame with Quote Data
    enriched_data = []
    
    for idx, row in option_chain_df.iterrows():
        # Reconstruct key used for lookup
        if 'exchange' in row:
            key = f"{row['exchange']}:{row['tradingsymbol']}"
        else:
            key = f"NFO:{row['tradingsymbol']}"
            
        quote = quotes.get(key, {})
        
        # Market Depth Extraction
        depth = quote.get('depth', {})
        buy_depth = depth.get('buy', [])
        sell_depth = depth.get('sell', [])
        
        # Create row dict and update
        item = row.to_dict()
        item.update({
            'ltp': quote.get('last_price', 0),
            'oi': quote.get('oi', 0),
            'volume': quote.get('volume', 0),
            'bid': buy_depth[0]['price'] if buy_depth else 0,
            'ask': sell_depth[0]['price'] if sell_depth else 0,
            'bid_qty': buy_depth[0]['quantity'] if buy_depth else 0,
            'ask_qty': sell_depth[0]['quantity'] if sell_depth else 0,
            'ohlc': quote.get('ohlc', {})
        })
        enriched_data.append(item)

    return pd.DataFrame(enriched_data)

print("✅ Updated build_option_chain and enrich_with_market_data to use lookup tables and optimized dataframes.")

✅ Updated build_option_chain and enrich_with_market_data to use lookup tables and optimized dataframes.


In [24]:
!pipenv shell
# Activate pipenv shell to ensure all dependencies are loaded

^C


In [29]:
from scipy.stats import norm

def get_underlying_ltp(kite, symbol, expiry, source_df=None):
    """
    Attempts to find the underlying price. 
    1. Looks for a Future contract expiring ON or AFTER the option expiry (typical for MCX).
    2. Fallback to Spot index/equity mapping.
    """
    underlying_ltp = 0
    used_symbol = ""

    # Ensure expiry is datetime
    if not isinstance(expiry, pd.Timestamp):
        try:f
            expiry = pd.to_datetime(expiry)
        except:
            pass

    # 1. Try to find corresponding Future for this expiry if dataframe provided
    if source_df is not None and not source_df.empty:
        # Find all Futures for this symbol
        mask_fut = (source_df['name'] == symbol) & (source_df['instrument_type'] == 'FUT')
        futures = source_df[mask_fut].copy()
        
        if not futures.empty:
            # Ensure future expiries are datetime
            # (Assuming source_df is standardized, but safety check)
            if 'expiry' in futures.columns: # extra safety
                 futures['expiry'] = pd.to_datetime(futures['expiry'])
            
            # Sort by expiry
            futures = futures.sort_values('expiry')
            
            # Filter: Future Expiry >= Option Expiry
            # Logic: Option settles into the nearest future that is active at option expiry
            valid_futures = futures[futures['expiry'] >= expiry]
            
            if not valid_futures.empty:
                # Pick the nearest valid future (First one)
                fut_row = valid_futures.iloc[0]
                
                tradingsymbol = fut_row['tradingsymbol']
                exch = fut_row['exchange'] 
                fut_expiry = fut_row['expiry']
                
                instrument_token = f"{exch}:{tradingsymbol}"
                try:
                    ltp_resp = kite.ltp(instrument_token)
                    if instrument_token in ltp_resp:
                        underlying_ltp = ltp_resp[instrument_token]['last_price']
                        used_symbol = instrument_token
                        print(f"🔹 Underlying identified as FUTURE: {used_symbol} (Exp: {fut_expiry.date()}) | Option Exp: {expiry.date()}")
                        return underlying_ltp
                except:
                    pass
            else:
                print(f"⚠️ No future found expiring on/after {expiry.date()}")

    # 2. Fallback to Spot (Indices or Stocks)
    index_map = {
        'NIFTY': 'NSE:NIFTY 50',
        'BANKNIFTY': 'NSE:NIFTY BANK',
        'FINNIFTY': 'NSE:NIFTY FIN SERVICE'
    }
    
    if symbol in index_map:
        used_symbol = index_map[symbol]
    else:
        # Assume it's a stock on NSE if not NIFTY/BANKNIFTY and Future lookup failed/wasn't applicable
        # This is a broad assumption; might need refinement for BSE
        used_symbol = f"NSE:{symbol}"
    
    try:
        ltp_resp = kite.ltp(used_symbol)
        if used_symbol in ltp_resp:
            underlying_ltp = ltp_resp[used_symbol]['last_price']
            print(f"🔹 Underlying identified as SPOT: {used_symbol} (LTP: {underlying_ltp})")
        else:
             print(f"⚠️ Could not fetch LTP for {used_symbol} (Spot)")
    except Exception as e:
        print(f"❌ Error fetching underlying {used_symbol}: {e}")
        
    return underlying_ltp


def implied_volatility(price, S, K, T, r, flag):
    """
    Finds implied volatility using Newton-Raphson method.
    """
    MAX_ITER = 100
    PRECISION = 1.0e-5
    sigma = 0.5 # Initial guess
    
    for i in range(MAX_ITER):
        # Calculate BS Price
        d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        
        if flag == 'CE':
            bs_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
            vega = S * np.sqrt(T) * norm.pdf(d1)
        else:
            bs_price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
            vega = S * np.sqrt(T) * norm.pdf(d1)
            
        diff = price - bs_price
        
        if abs(diff) < PRECISION:
            return sigma
        
        if abs(vega) < 1e-8: # Avoid division by zero
             return sigma # Return current best guess or 0.0
             
        sigma = sigma + (diff / vega)
        
    return sigma

def add_iv_columns(chain, underlying_ltp, risk_free_rate=0.10):
    ivs = []
    today = pd.Timestamp.now()
    
    for _, row in chain.iterrows():
        try:
            # Inputs
            price = row['ltp']
            strike = row['strike']
            expiry = row['expiry']
            # If price is 0, IV is 0
            if price <= 0:
                ivs.append(0.0)
                continue

            # Time to expiry
            if not isinstance(expiry, pd.Timestamp):
                expiry = pd.to_datetime(expiry)
                
            # Set time to end of expiry day (approx)
            expiry_end = expiry + pd.Timedelta(hours=15, minutes=30)
            
            # Difference in years
            delta = expiry_end - today
            T = delta.total_seconds() / (365.0 * 24 * 3600)
            
            if T <= 0.001: # Expiring now/expired
                T = 0.001
            
            iv = implied_volatility(
                price, 
                underlying_ltp, 
                strike, 
                T, 
                risk_free_rate, 
                row['instrument_type']
            )
            ivs.append(iv * 100) # Percentage
        except Exception:
            ivs.append(0.0)
            
    chain['iv'] = ivs
    return chain

print("✅ IV Calculation Functions (Black-Scholes) added.")

✅ IV Calculation Functions (Black-Scholes) added.


   ---------------------------------------- 0.0/36.3 MB ? eta -:--:--
    --------------------------------------- 0.8/36.3 MB 4.2 MB/s eta 0:00:09
   -- ------------------------------------- 2.1/36.3 MB 4.9 MB/s eta 0:00:07
   --- ------------------------------------ 3.4/36.3 MB 5.4 MB/s eta 0:00:07
   ----- ---------------------------------- 4.7/36.3 MB 5.8 MB/s eta 0:00:06
   ------- -------------------------------- 6.6/36.3 MB 6.1 MB/s eta 0:00:05
   -------- ------------------------------- 7.9/36.3 MB 6.3 MB/s eta 0:00:05
   ---------- ----------------------------- 9.7/36.3 MB 6.4 MB/s eta 0:00:05
   ------------ --------------------------- 11.5/36.3 MB 6.7 MB/s eta 0:00:04
   -------------- ------------------------- 13.4/36.3 MB 7.0 MB/s eta 0:00:04
   ----------------- ---------------------- 15.5/36.3 MB 7.2 MB/s eta 0:00:03
   ------------------ --------------------- 16.5/36.3 MB 7.0 MB/s eta 0:00:03
   -------------------- ------------------- 18.4/36.3 MB 7.2 MB/s eta 0:00:03
 


[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
# Test kite.quote() with a list of symbols
symbols = ["NIFTY26FEB24000CE", "NIFTY26FEB24000PE", "NIFTY26FEB23900CE"]

quotes = kite.quote(symbols)
pprint(quotes)

{}


In [21]:
# Test the new functions and IV Calculation
symbol = "GOLDM"

# 1. Get available expiries
available_expiries = get_all_expiries(kite, symbol)

if len(available_expiries) > 0:
    test_expiry = available_expiries[0]
    print(f"Testing with Symbol: {symbol}, Expiry: {test_expiry.date()}")
    
    # 2. Build and enrich chain
    chain = build_option_chain(symbol, test_expiry)
    chain = enrich_with_market_data(kite, chain)

    if not chain.empty:
        # 3. Determine Source Dataframe for Futures lookup
        exch = chain.iloc[0]['exchange'] if 'exchange' in chain.columns else 'NFO'
        source_df_for_ltp = df_mcx if exch == 'MCX' and 'df_mcx' in globals() else (df_all if 'df_all' in globals() else None)
        
        # 4. Get Underlying LTP
        underlying_ltp = get_underlying_ltp(kite, symbol, test_expiry, source_df_for_ltp)
        
        if underlying_ltp > 0:
            # 5. Calculate IV
            chain = add_iv_columns(chain, underlying_ltp)
            print(f"✅ IV Calculated using Underlying LTP: {underlying_ltp}")
            
            # 6. Show Results (Filter interesting columns)
            cols = ['tradingsymbol', 'strike', 'instrument_type', 'ltp', 'iv', 'oi', 'volume']
            display_cols = [c for c in cols if c in chain.columns]
            
            # Show ATMish options (around underlying price)
            chain['diff_pct'] = abs(chain['strike'] - underlying_ltp) / underlying_ltp
            atm_view = chain.sort_values('diff_pct').head(10).sort_values('strike')
            
            print("\n📊 Option Chain with IV (ATM View):")
            print(atm_view[display_cols].to_string(index=False))
        else:
            print("❌ Skipping IV calculation (Underlying LTP not found)")
            print(chain[['tradingsymbol', 'strike', 'ltp']].head())
            
    else:
        print("Chain is empty for this expiry.")
else:
    print(f"❌ No expiries found for {symbol}.")

Testing with Symbol: GOLDM, Expiry: 2026-02-26
Error fetching quotes: Unknown Content-Type (text/html) with response: (b'<html>\r\n<head><title>414 Request-URI Too Large</title></head>\r\n<body>\r\n<center><h1>414 Request-URI Too Large</h1></center>\r\n</body>\r\n</html>\r\n')

✅ Fetched 1130 option contracts.
        tradingsymbol    strike instrument_type
0  GOLDM26FEB126200PE  126200.0              PE
1  GOLDM26FEB126200CE  126200.0              CE
2  GOLDM26FEB126300CE  126300.0              CE
3  GOLDM26FEB126300PE  126300.0              PE
4  GOLDM26FEB126400CE  126400.0              CE


In [20]:
symbol= 'GOLDM'
test_expiry = get_all_expiries(kite, symbol)[1]

chain = build_option_chain(symbol, test_expiry)
chain = enrich_with_market_data(kite, chain)

Error fetching quotes: Unknown Content-Type (text/html) with response: (b'<html>\r\n<head><title>414 Request-URI Too Large</title></head>\r\n<body>\r\n<center><h1>414 Request-URI Too Large</h1></center>\r\n</body>\r\n</html>\r\n')


In [31]:
# Calculate IV for the existing 'chain' dataframe
# Using the global 'chain' variable from previous generation

if 'chain' in locals() and not chain.empty and 'symbol' in locals():
    print(f"Running IV Calculation for symbol: {symbol}...")
    
    # 1. Identify Expiry and Exchange from the chain data itself
    chain_expiry = chain['expiry'].iloc[0]
    exchange = chain['exchange'].iloc[0] if 'exchange' in chain.columns else 'NFO'
    
    print(f"   Expiry: {chain_expiry} | Exchange: {exchange}")

    # 1.5 Ensure Market Data (LTP) is present
    if 'ltp' not in chain.columns:
        print("⚠️ Market data (LTP) missing from chain. Fetching quotes now...")
        chain = enrich_with_market_data(kite, chain)
        if 'ltp' not in chain.columns:
             print("❌ Failed to fetch market data. Aborting IV calculation.")
             # Stop usage of chain here if critical
    
    # 2. Select Source DataFrame for Future Price lookup
    # (MCX futures are in df_mcx, NFO futures in df_all)
    source_df = None
    if exchange == 'MCX' and 'df_mcx' in globals():
        source_df = df_mcx
    elif 'df_all' in globals():
        source_df = df_all
        
    # 3. Fetch Underlying LTP (Spot or Future)
    underlying_ltp = get_underlying_ltp(kite, symbol, chain_expiry, source_df)
    
    if underlying_ltp > 0 and 'ltp' in chain.columns:
        # 4. Apply Black-Scholes IV Calculation
        chain = add_iv_columns(chain, underlying_ltp)
        print(f"✅ IV Calculated successfully! Underlying Ref Price: {underlying_ltp}")
        
        # 5. Display ATM Options (formatted)
        # Find ATM by smallest distance to underlying price
        chain['diff_pct'] = abs(chain['strike'] - underlying_ltp) / underlying_ltp
        atm_view = chain.sort_values('diff_pct').head(10).sort_values('strike')
        
        cols = ['tradingsymbol', 'strike', 'instrument_type', 'ltp', 'iv', 'oi', 'volume']
        display_cols = [c for c in cols if c in chain.columns]
        
        print("\n📊 Option Chain with IV (ATM View):")
        print(atm_view[display_cols].to_string(index=False))
        
    else:
        print(f"❌ Failed to get Underlying LTP or Market Data. Cannot calculate IV.")
else:
    print("❌ 'chain' dataframe or 'symbol' variable is missing. Please run the chain generation cell above.")

Running IV Calculation for symbol: GOLDM...
   Expiry: 2026-02-26 00:00:00 | Exchange: MCX
⚠️ Market data (LTP) missing from chain. Fetching quotes now...
Error fetching quotes: Unknown Content-Type (text/html) with response: (b'<html>\r\n<head><title>414 Request-URI Too Large</title></head>\r\n<body>\r\n<center><h1>414 Request-URI Too Large</h1></center>\r\n</body>\r\n</html>\r\n')
❌ Failed to fetch market data. Aborting IV calculation.
🔹 Underlying identified as FUTURE: MCX:GOLDM26MARFUT (Exp: 2026-03-05) | Option Exp: 2026-02-26
❌ Failed to get Underlying LTP or Market Data. Cannot calculate IV.


In [28]:
# DEBUG: Inspect MCX Futures for GOLDM to fix underlying lookup
print("DEBUG: Checking MCX Futures for GOLDM")
if 'df_mcx' in globals():
    mcx_futs = df_mcx[(df_mcx['name'] == 'GOLDM') & (df_mcx['instrument_type'] == 'FUT')]
    print(mcx_futs[['tradingsymbol', 'expiry', 'instrument_type']].sort_values('expiry'))
    
    print(f"\nTarget Expiry from Option Chain: {chain['expiry'].iloc[0]}")
else:
    print("df_mcx not found")

DEBUG: Checking MCX Futures for GOLDM
    tradingsymbol     expiry instrument_type
68  GOLDM26FEBFUT 2026-02-05             FUT
69  GOLDM26MARFUT 2026-03-05             FUT
70  GOLDM26APRFUT 2026-04-03             FUT
71  GOLDM26MAYFUT 2026-05-05             FUT
72  GOLDM26JUNFUT 2026-06-05             FUT
73  GOLDM26JULFUT 2026-07-03             FUT

Target Expiry from Option Chain: 2026-02-26 00:00:00


In [22]:
chain

,tradingsymbol,strike,instrument_type,expiry,lot_size,exchange,instrument_token
0,GOLDM26FEB126200PE,126200.0,PE,2026-02-26,1,MCX,125991175
1,GOLDM26FEB126200CE,126200.0,CE,2026-02-26,1,MCX,125989895
2,GOLDM26FEB126300CE,126300.0,CE,2026-02-26,1,MCX,125989639
3,GOLDM26FEB126300PE,126300.0,PE,2026-02-26,1,MCX,125990919
4,GOLDM26FEB126400CE,126400.0,CE,2026-02-26,1,MCX,125989383
...,...,...,...,...,...,...,...
1125,GOLDM26FEB204000CE,204000.0,CE,2026-02-26,1,MCX,141098503
1126,GOLDM26FEB204500PE,204500.0,PE,2026-02-26,1,MCX,141127687
1127,GOLDM26FEB204500CE,204500.0,CE,2026-02-26,1,MCX,141098759
1128,GOLDM26FEB205000PE,205000.0,PE,2026-02-26,1,MCX,141127943


## 8. Position Summary Analysis

In [56]:
pos_day = pd.DataFrame(positions['day'])
pos_net = pd.DataFrame(positions['net'])
pos_combined = pd.concat([pos_day, pos_net], ignore_index=True)

# 2. Fix Missing Columns (Safety Check)
# If 'expiry' or 'strike' don't exist, create them as empty strings
for col in ['expiry', 'strike', 'instrument_type']:
    if col not in pos_combined.columns:
        pos_combined[col] = ""

# 3. Clean up Expiry (Only if it contains data)
pos_combined['expiry'] = pd.to_datetime(pos_combined['expiry'], errors='coerce')

# 4. Create the "Goddamn" Summary
# We include expiry, strike, and type for a complete breakdown
pos_summary = pos_combined.groupby(['tradingsymbol', 'expiry', 'strike', 'instrument_type']).agg({
    'quantity': 'sum',
    'pnl': 'sum',
    'average_price': 'mean',
    'last_price': 'first',
    'm2m': 'sum'
}).reset_index()

# 5. Add Key Stats
# Exposure value (Market Value)
pos_summary['exposure'] = pos_summary['quantity'] * pos_summary['last_price']

# P&L Percentage (Safety check for 0 average price)
pos_summary['pnl_pct'] = 0.0
mask = (pos_summary['average_price'] != 0) & (pos_summary['quantity'] != 0)
pos_summary.loc[mask, 'pnl_pct'] = (pos_summary['pnl'] / (pos_summary['average_price'] * pos_summary['quantity'].abs())) * 100

# 6. Save to Parquet
pos_summary.to_parquet('pos_xyz_summary.parquet', engine='pyarrow')

print("Summary Generated Successfully!")
print(pos_summary[['tradingsymbol', 'pnl', 'pnl_pct', 'exposure']].head())

Summary Generated Successfully!
Empty DataFrame
Columns: [tradingsymbol, pnl, pnl_pct, exposure]
Index: []


## 9. Group Positions by Base Symbol and Expiry

In [57]:
# Group positions by base symbol (name) and expiry, then merge with master df_all

# Step 1: Merge positions with df_all to get the 'name' (base symbol)
pos_enriched = pos_combined.merge(
    df_all[['tradingsymbol', 'name', 'expiry', 'instrument_type', 'strike', 'lot_size']],
    on='tradingsymbol',
    how='left',
    suffixes=('', '_master')
)

# Use the master expiry if position expiry is missing
if 'expiry_master' in pos_enriched.columns:
    pos_enriched['expiry'] = pos_enriched['expiry'].fillna(pos_enriched['expiry_master'])

# Step 2: Group by base name and expiry
position_groups = pos_enriched.groupby(['name', 'expiry']).agg({
    'tradingsymbol': 'count',  # Count of positions
    'quantity': 'sum',
    'pnl': 'sum',
    'm2m': 'sum',
    'average_price': 'mean',
    'last_price': 'mean',
    'instrument_type': lambda x: x.unique().tolist()  # List unique instrument types
}).reset_index()

# Rename for clarity
position_groups.rename(columns={'tradingsymbol': 'position_count'}, inplace=True)

# Calculate total exposure
position_groups['total_exposure'] = position_groups['quantity'] * position_groups['last_price']

print("="*80)
print("POSITIONS GROUPED BY BASE SYMBOL AND EXPIRY")
print("="*80)
print(position_groups)
print(f"\nTotal unique base symbols with positions: {position_groups['name'].nunique()}")
print(f"Total position groups: {len(position_groups)}")

POSITIONS GROUPED BY BASE SYMBOL AND EXPIRY
          name     expiry  position_count  quantity           pnl  m2m  \
0     ADANIENT 2026-02-24               1      -309   6952.500000    0   
1   ADANIGREEN 2026-02-24               2     -1200   1350.000000    0   
2     HDFCBANK 2026-02-24               1     -1100  -1842.500000    0   
3          IEX 2026-02-24               2     -7500    -37.500000    0   
4         INFY 2026-02-24               2      -800    420.000000    0   
5          ITC 2026-02-24               2     -8000  -7760.001600    0   
6   KALYANKJIL 2026-02-24               2     -3525 -16920.000000    0   
7   MIDCPNIFTY 2026-02-24               1      -120   -132.000000    0   
8        NIFTY 2026-02-10               1        65    143.000000    0   
9        NIFTY 2026-02-24               1        65   -949.000000    0   
10       NIFTY 2026-03-30               3      -195 -20751.250065    0   
11   TATAELXSI 2026-02-24               2      -200  22400.000000   

## 10. Detailed Position View by Base Symbol

In [58]:
# Detailed view: Show all positions for each base symbol grouped by expiry

print("\n" + "="*80)
print("DETAILED POSITIONS BY BASE SYMBOL")
print("="*80)

for name in pos_enriched['name'].unique():
    name_positions = pos_enriched[pos_enriched['name'] == name].copy()
    
    # Sort by expiry and instrument type
    name_positions = name_positions.sort_values(['expiry', 'instrument_type_master'])
    
    print(f"\n{'='*80}")
    print(f"📊 {name}")
    print(f"{'='*80}")
    
    # Show key columns
    display_cols = ['tradingsymbol', 'instrument_type_master', 'expiry', 'strike_master', 
                    'quantity', 'average_price', 'last_price', 'pnl', 'm2m']
    
    # Filter to existing columns
    display_cols = [col for col in display_cols if col in name_positions.columns]
    
    print(name_positions[display_cols].to_string(index=False))
    
    # Summary for this symbol
    total_qty = name_positions['quantity'].sum()
    total_pnl = name_positions['pnl'].sum()
    total_m2m = name_positions['m2m'].sum()
    
    print(f"\n  📈 Total Quantity: {total_qty}")
    print(f"  💰 Total P&L: ₹{total_pnl:,.2f}")
    print(f"  📊 Total M2M: ₹{total_m2m:,.2f}")


DETAILED POSITIONS BY BASE SYMBOL

📊 nan
Empty DataFrame
Columns: [tradingsymbol, instrument_type_master, expiry, strike_master, quantity, average_price, last_price, pnl, m2m]
Index: []

  📈 Total Quantity: 0
  💰 Total P&L: ₹0.00
  📊 Total M2M: ₹0.00

📊 ADANIENT
      tradingsymbol instrument_type_master     expiry  strike_master  quantity  average_price  last_price    pnl  m2m
ADANIENT26FEB1800PE                     PE 2026-02-24         1800.0      -309          52.65       30.15 6952.5    0

  📈 Total Quantity: -309
  💰 Total P&L: ₹6,952.50
  📊 Total M2M: ₹0.00

📊 ADANIGREEN
       tradingsymbol instrument_type_master     expiry  strike_master  quantity  average_price  last_price    pnl  m2m
ADANIGREEN26FEB900CE                     CE 2026-02-24          900.0      -600          17.40       15.05 1410.0    0
ADANIGREEN26FEB620PE                     PE 2026-02-24          620.0      -600           3.55        3.65  -60.0    0

  📈 Total Quantity: -1200
  💰 Total P&L: ₹1,350.00
  📊 T

## Red/Green/Yellow Position Classification

## 11. Classify Positions (Red/Green/Yellow)

In [ ]:
# Classify positions as Red (losing), Green (profit), or Yellow (breakeven/small moves)
# Red: P&L < -1000
# Green: P&L > 1000
# Yellow: -1000 <= P&L <= 1000

# Create masks for classification
mask_red = pos_combined['pnl'] < -1000
mask_green = pos_combined['pnl'] > 1000
mask_yellow = (pos_combined['pnl'] >= -1000) & (pos_combined['pnl'] <= 1000)

# Create the dataframes
df_red = pos_combined[mask_red].copy()
df_green = pos_combined[mask_green].copy()
df_yellow = pos_combined[mask_yellow].copy()

print("="*80)
print("POSITION CLASSIFICATION BY P&L")
print("="*80)
print(f"\n🔴 RED (Loss > ₹1000): {len(df_red)} positions")
print(f"   Total P&L: ₹{df_red['pnl'].sum():,.2f}")
if len(df_red) > 0:
    print(f"   Avg P&L: ₹{df_red['pnl'].mean():,.2f}")
    print(f"   Worst: ₹{df_red['pnl'].min():,.2f}")

print(f"\n🟡 YELLOW (Between -₹1000 and +₹1000): {len(df_yellow)} positions")
print(f"   Total P&L: ₹{df_yellow['pnl'].sum():,.2f}")
if len(df_yellow) > 0:
    print(f"   Avg P&L: ₹{df_yellow['pnl'].mean():,.2f}")

print(f"\n🟢 GREEN (Profit > ₹1000): {len(df_green)} positions")
print(f"   Total P&L: ₹{df_green['pnl'].sum():,.2f}")
if len(df_green) > 0:
    print(f"   Avg P&L: ₹{df_green['pnl'].mean():,.2f}")
    print(f"   Best: ₹{df_green['pnl'].max():,.2f}")

print(f"\n{'='*80}")
print(f"OVERALL SUMMARY")
print(f"{'='*80}")
print(f"Total Positions: {len(pos_combined)}")
print(f"Net P&L: ₹{pos_combined['pnl'].sum():,.2f}")

## 12. Display Red Positions (Losses)

In [ ]:
# Display red positions in detail
if len(df_red) > 0:
    print("\n" + "="*80)
    print("🔴 RED POSITIONS (Losses > ₹1000)")
    print("="*80)
    display_cols = ['tradingsymbol', 'quantity', 'average_price', 'last_price', 'pnl', 'm2m']
    display_cols = [col for col in display_cols if col in df_red.columns]
    print(df_red[display_cols].sort_values('pnl').to_string(index=False))

## 13. Display Green Positions (Profits)

In [ ]:
# Display green positions in detail
if len(df_green) > 0:
    print("\n" + "="*80)
    print("🟢 GREEN POSITIONS (Profits > ₹1000)")
    print("="*80)
    display_cols = ['tradingsymbol', 'quantity', 'average_price', 'last_price', 'pnl', 'm2m']
    display_cols = [col for col in display_cols if col in df_green.columns]
    print(df_green[display_cols].sort_values('pnl', ascending=False).to_string(index=False))

## 14. SQLite Database Setup

In [ ]:
import sqlite3
import os

# Create a SQLite database (creates file if it doesn't exist)
db_path = "trading_data.db"

# Connect to the database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

print(f"Connected to SQLite database: {os.path.abspath(db_path)}")

Connected to SQLite database: c:\Users\sathy\OneDrive\Desktop\Project Algoarms\pykite\trading_data.db


## 15. Create Database Tables

In [ ]:
# Create a sample table for storing trades
cursor.execute('''
    CREATE TABLE IF NOT EXISTS trades (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        symbol TEXT NOT NULL,
        trade_type TEXT NOT NULL,
        quantity INTEGER NOT NULL,
        price REAL NOT NULL,
        timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
    )
''')

# Create a table for storing holdings snapshots
cursor.execute('''
    CREATE TABLE IF NOT EXISTS holdings_snapshot (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        symbol TEXT NOT NULL,
        quantity INTEGER,
        average_price REAL,
        last_price REAL,
        pnl REAL,
        snapshot_time DATETIME DEFAULT CURRENT_TIMESTAMP
    )
''')

conn.commit()
print("Tables created successfully!")

Tables created successfully!


## 16. Insert Sample Trade Data

In [ ]:
# Example: Insert a sample trade
cursor.execute('''
    INSERT INTO trades (symbol, trade_type, quantity, price)
    VALUES (?, ?, ?, ?)
''', ('NSE:RELIANCE', 'BUY', 10, 2450.50))

conn.commit()

# Query all trades
cursor.execute('SELECT * FROM trades')
trades = cursor.fetchall()
print("All trades:")
for trade in trades:
    print(trade)

All trades:
(1, 'NSE:RELIANCE', 'BUY', 10, 2450.5, '2026-01-29 00:46:12')
(2, 'NSE:RELIANCE', 'BUY', 10, 2450.5, '2026-01-29 02:02:52')


## 17. Database Helper Functions

In [ ]:
# Helper function to close the database connection
def close_db():
    conn.close()
    print("Database connection closed.")

# Uncomment to close when done:
# close_db()

## 18. Get LTP and Quotes

In [ ]:
# Get LTP (Last Traded Price) for a single stock
ltp = kite.ltp("NSE:RELIANCE")
print(ltp)

# Or get LTP for multiple instruments
ltps = kite.ltp(["NSE:RELIANCE", "NSE:INFY", "NSE:TCS"])
print(ltps)

# Get detailed quote with more data
quote = kite.quote("NSE:RELIANCE")
print(quote)

# Get multiple quotes
quotes = kite.quote(["NSE:RELIANCE", "NSE:INFY"])
print(quotes)


PermissionException: Insufficient permission for that call.

## 19. API Rate Limit Test

In [61]:
import time
from datetime import datetime
from IPython.display import clear_output

# API Rate Limit Test
counter = 1
try:
    while True:
        start_time = time.time()
        
        # Make API calls
        positions = kite.positions()
        orders = kite.orders()
        
        end_time = time.time()
        elapsed = end_time - start_time
        
        # Clear previous output and print current stats
        clear_output(wait=True)
        print(f"Request #{counter}")
        print(f"Time: {datetime.now().strftime('%H:%M:%S')}")
        print(f"Response Time: {elapsed:.3f} seconds")
        print(f"Positions retrieved: {len(positions.get('net', []))} net, {len(positions.get('day', []))} day")
        print(f"Orders retrieved: {len(orders)}")
        print("\nPress Ctrl+C to stop...")
        
        counter += 1
        
        # Sleep for 0.1 second before next request
        time.sleep(0.1)
        
except KeyboardInterrupt:
    print(f"\n\nStopped after {counter-1} requests")
except Exception as e:
    print(f"\n\nError occurred after {counter-1} requests:")
    print(f"Error Type: {type(e).__name__}")
    print(f"Error Message: {str(e)}")
    print(f"Time: {datetime.now().strftime('%H:%M:%S')}")

Request #45
Time: 03:34:52
Response Time: 0.446 seconds
Positions retrieved: 35 net, 0 day
Orders retrieved: 0

Press Ctrl+C to stop...


Stopped after 45 requests
